In [ ]:
%%time
'''In google colab'''
#Update local version of the package catalog t enable installation of related packages.
!apt update
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark 3.2.1
!wget -q https://apache.osuosl.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
# unzip spark files
!tar xf spark-3.2.4-bin-hadoop3.2.tgz
# install findspark 
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop3.2"

import findspark 
findspark.init() 

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import *
# Set up PySpark session
spark = SparkSession.builder.appName("Big Data Analytics").getOrCreate()

In [8]:
import tweepy
import datetime
import pandas as pd
import sys

# credentials from https://apps.twitter.com/

consumer_key = "meyT3EhfyNfwQAANiSIIwj3Pp"
consumer_secret = "5OG1PdhayFAUGGGsajMwbi5xImKLLTk0m0yiAAU2MiVMUxHkjP"
access_token = "818345262221590528-2eMomHsHRBjzPYZvPF5Lo4zUSrAug33"
access_token_secret = "rXNEDYEHudCrx9N1IwMwJ2o0c0hVYFcdDocz4yQgq7G6k"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

# Retrieve tweets
keywords = ["AI", "Artificial Intelligence", "artificialintelligence"]
def download_tweets(search_query, num_tweets):
    # Initialize list to store downloaded tweets
    tweets = []

    # Download tweets using Twitter API search
    api = tweepy.API(auth, wait_on_rate_limit=True)
    for tweet in tweepy.Cursor(api.search_tweets, q=search_query, lang="en", tweet_mode="extended").items(num_tweets):
        #print(tweet)
        tweets.append(tweet._json)

    return pd.DataFrame(tweets)
tweets = download_tweets(keywords[1], 8000)

c:\users\monga\appdata\local\programs\python\python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\monga\appdata\local\programs\python\python38\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\users\monga\appdata\local\programs\python\python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\users\monga\appdata\local\programs\python\python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [160]:
#pip install --upgrade sqlalchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 21.6 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.48
    Uninstalling SQLAlchemy-1.4.48:
      Successfully uninstalled SQLAlchemy-1.4.48
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databases 0.7.0 requires sqlalchemy<1.5,>=1.4.42, but you have sqlalchemy 2.0.13 which is incompatible.


In [163]:
from sqlalchemy import create_engine
# create a database connection
engine = create_engine('sqlite:///ArtificialIntelligenceTweets.db')
# write the DataFrame to a SQL database
tweets[['created_at',  'full_text',  'retweet_count', 'favorite_count',
       'favorited']].to_sql('AITwitterData', con=engine, index = False)

In [ ]:
data = pd.read_sql('AITwitterData', con  = engine)

In [218]:
from pyspark.sql.functions import explode, col, when
hashtags_df = df.select(explode(col("entities")),
                        when(col("entities").isNotNull(), 1).otherwise(0).alias("hashtag_present"))

In [210]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
tweets1 = spark.createDataFrame(tweets[['created_at',  'full_text',  'retweet_count', 'favorite_count',
       'favorited',  'retweeted', 'in_reply_to_screen_name', 'entities']])

/content/spark-3.2.4-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [199]:
tweets.columns

Index(['created_at', 'id', 'id_str', 'full_text', 'truncated',
       'display_text_range', 'entities', 'metadata', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'possibly_sensitive', 'lang',
       'retweeted_status', 'extended_entities', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status'],
      dtype='object')

In [167]:
tweets = pd.read_sql('AITwitterData', con  = engine)

In [280]:
from pyspark.sql.functions import regexp_replace, lower, split
df = tweets1
# Clean the text column
df = df.withColumn("tweet", regexp_replace("full_text", "[^a-zA-Z\\s]", ""))  # Remove special characters
df = df.withColumn("tweet", lower("tweet"))  # Convert text to lowercase
from pyspark.sql.functions import explode, col
hashtags_df = df.select(explode(col("entities")),
                        when(col("entities").isNotNull(), 1).otherwise(0).alias("hashtags"))

hashtags_df.show()

+-------------+--------------------+--------+
|          key|               value|hashtags|
+-------------+--------------------+--------+
|         urls|[{expanded_url ->...|       1|
|     hashtags|                  []|       1|
|user_mentions|                  []|       1|
|      symbols|                  []|       1|
|         urls|                  []|       1|
|     hashtags|                  []|       1|
|user_mentions|[{name -> SgtButa...|       1|
|      symbols|                  []|       1|
|         urls|[{expanded_url ->...|       1|
|     hashtags|                  []|       1|
|user_mentions|                  []|       1|
|      symbols|                  []|       1|
|         urls|[{expanded_url ->...|       1|
|     hashtags|                  []|       1|
|user_mentions|[{name -> Wayne, ...|       1|
|      symbols|                  []|       1|
|         urls|                  []|       1|
|        media|[{display_url -> ...|       1|
|     hashtags|[{indices -> [25,..

In [281]:
tweets1.createOrReplaceTempView('users')
#check database
spark.sql('SELECT * FROM users LIMIT 2').toPandas()

,created_at,full_text,retweet_count,favorite_count,favorited,retweeted,in_reply_to_screen_name,entities
0,Sun May 14 22:35:03 +0000 2023,The role of artificial intelligence in marketi...,0,0,False,False,None,{'urls': [{'expanded_url': 'https://bit.ly/3LO...
1,Sun May 14 22:35:03 +0000 2023,"@SgtButane @lenphil29 Yes, but Artificial Inte...",0,0,False,False,SgtButane,"{'urls': [], 'hashtags': [], 'user_mentions': ..."


In [282]:
query = '''
SELECT in_reply_to_screen_name, favorite_count
FROM users
ORDER BY favorite_count DESC
'''
spark.sql(query).show()

+-----------------------+--------------+
|in_reply_to_screen_name|favorite_count|
+-----------------------+--------------+
|                   null|            42|
|                   null|            37|
|                   null|            23|
|           PapiRichie12|            22|
|                   null|            17|
|                   null|            14|
|                   null|            13|
|                   null|            12|
|           BeastofEarth|            11|
|          ListingsAlert|            10|
|                   null|             9|
|                   null|             8|
|                   null|             8|
|                   null|             8|
|                   null|             8|
|                   null|             7|
|                   null|             7|
|                   null|             7|
|                   null|             6|
|                   null|             6|
+-----------------------+--------------+
only showing top

#### Sentiment Analysis

In [283]:
%%time
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from textblob import TextBlob

# Define a sentiment analysis function using TextBlob
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

# Create a user-defined function (udf) for the sentiment analysis function
sentiment_udf = udf(get_sentiment, FloatType())

# Apply the sentiment analysis udf to the cleaned text column
df = df.withColumn("sentiment", sentiment_udf(df["tweet"]))
#map the scores
def get_emotion(score):
    if score > 0.2:
        return "positive"
    elif score < -0.2:
        return "negative"
    else:
        return "neutral"
# function (udf) for the emotion function
emotion_udf = udf(get_emotion, StringType())

# Apply the emotion udf to the sentiment column
df = df.withColumn("target", emotion_udf(df["sentiment"]))
#results
df.show()

+--------------------+--------------------+-------------+--------------+---------+---------+-----------------------+--------------------+--------------------+-----------+--------+
|          created_at|           full_text|retweet_count|favorite_count|favorited|retweeted|in_reply_to_screen_name|            entities|               tweet|  sentiment|  target|
+--------------------+--------------------+-------------+--------------+---------+---------+-----------------------+--------------------+--------------------+-----------+--------+
|Sun May 14 22:35:...|The role of artif...|            0|             0|    false|    false|                   null|{urls -> [{expand...|the role of artif...|       -0.6|negative|
|Sun May 14 22:35:...|@SgtButane @lenph...|            0|             0|    false|    false|              SgtButane|{urls -> [], hash...|sgtbutane lenphil...|-0.14444445| neutral|
|Sun May 14 22:35:...|New Blog Post fro...|            0|             0|    false|    false|        

#### Logistic Regression

In [284]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StringIndexer, CountVectorizer, NGram, VectorAssembler, ChiSqSelector

tokenizer = Tokenizer(inputCol="tweet", outputCol="words")

In [285]:
def build_trigrams(inputCol=["tweet","target"], n=3):
    
    # Create a Tokenizer object to tokenize inputCol and output to words column
    tokenizer = [Tokenizer(inputCol="tweet", outputCol="words")]
    
    # Create a list of NGram objects, with ngrams from 1 to n, and output to {i}_grams column
    ngrams = [NGram(n=i, inputCol="words", outputCol="{0}_grams".format(i))
        for i in range(1, n + 1)
    ]

    # Create a list of CountVectorizer objects, with vocabSize 2**14, and output to {i}_tf column
    cv = [
        CountVectorizer(vocabSize=2**14,inputCol="{0}_grams".format(i),
            outputCol="{0}_tf".format(i))
        for i in range(1, n + 1)
    ]
    
    # Create a list of IDF objects, and output to {i}_tfidf column
    idf = [IDF(inputCol="{0}_tf".format(i), outputCol="{0}_tfidf".format(i), minDocFreq=5) for i in range(1, n + 1)]

    # Create a VectorAssembler object to combine {i}_tfidf columns into a single rawFeatures column
    assembler = [VectorAssembler(
        inputCols=["{0}_tfidf".format(i) for i in range(1, n + 1)],
        outputCol="rawFeatures"
    )]
    
    # Create a StringIndexer object to convert target column to numeric label column
    label_stringIdx = [StringIndexer(inputCol = "target", outputCol = "label")]
    
    # Create a ChiSqSelector object to select top features from rawFeatures column, and output to features column
    selector = [ChiSqSelector(numTopFeatures=2**14,featuresCol='rawFeatures', outputCol="features")]
    
    # Create a LogisticRegression object to fit the model
    lr = [LogisticRegression()]
    
    # Combine all stages into a single Pipeline object and return
    return Pipeline(stages=tokenizer + ngrams + cv + idf + assembler + label_stringIdx + selector + lr)


In [286]:
%%time
#split data to 
# Split the data into training and test sets
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)
model = pipeline.fit(train_data)

predictions = model.transform(test_data)
from pyspark.ml.evaluation import BinaryClassificationEvaluator

predictions = model.transform(test_data)

CPU times: user 209 ms, sys: 26.2 ms, total: 235 ms
Wall time: 21.8 s


In [287]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % (accuracy))

Accuracy = 0.837989


#### Dashboard

In [291]:
datax.columns

Index(['created_at', 'full_text', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'in_reply_to_screen_name', 'entities',
       'tweet', 'sentiment', 'target', 'date'],
      dtype='object')

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
datax = df.toPandas()

# Calculate summary statistics for Sentiment tab
sentiment_summary = datax.groupby(['target']).size().reset_index(name='count')
sentiment_pie_chart = px.pie(sentiment_summary, values='count', names='target', hole=.3)

# Calculate summary statistics for Data Overview tab
data_overview_summary = datax.describe()

# Calculate time series for average polarity over time for Data Overview tab
datax['created_at'] = pd.to_datetime(datax['created_at'])
datax['date'] = datax['created_at'].dt.date
avg_polarity_time_series = datax.groupby(['date'])['sentiment'].mean().reset_index()
avg_polarity_time_series_chart = px.line(avg_polarity_time_series, x='date', y='sentiment')

# Create the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    
    # Define the tabs
    dcc.Tabs(id='tabs', value='sentiment-tab', children=[
        
        # Define the Sentiment tab
        dcc.Tab(label='Sentiment', value='sentiment-tab', children=[
            html.H1('Sentiment Summary'),
            dcc.Graph(figure=sentiment_pie_chart)
        ]),
        
        # Define the Data Overview tab
        dcc.Tab(label='Data Overview', value='data-overview-tab', children=[
            html.H1('Data Overview Summary'),
            html.Table([
                html.Thead(html.Tr([html.Th(col) for col in data_overview_summary.columns])),
                html.Tbody([
                    html.Tr([
                        html.Td(data_overview_summary.iloc[i][col]) for col in data_overview_summary.columns
                    ]) for i in range(len(data_overview_summary))
                ])
            ]),
            html.H1('Average Polarity Over Time'),
            dcc.Graph(figure=avg_polarity_time_series_chart)
        ])
    ])
])

if __name__ == '__main__':
    app.run_server(debug=False, )
